In [1]:
import time

import matplotlib.pyplot as plt
import monai
import numpy as np
import torch
from monai.config import print_config
from monai.transforms import ScaleIntensityd
from tqdm import tqdm

print_config()
from dataset import HNTSDataset

dataset = HNTSDataset("data/train", monai.transforms.Compose([
    monai.transforms.ScaleIntensityd(keys='image')
]))

D:\repos\tdt17_mini_project\venv\lib\site-packages\ignite\handlers\checkpoint.py:17: DeprecationWarning: `TorchScript` support for functional optimizers is deprecated and will be removed in a future PyTorch release. Consider using the `torch.compile` optimizer instead.
  from torch.distributed.optim import ZeroRedundancyOptimizer


MONAI version: 1.4.0
Numpy version: 1.26.4
Pytorch version: 2.5.1+cu124
MONAI flags: HAS_EXT = False, USE_COMPILED = False, USE_META_DICT = False
MONAI rev id: 46a5272196a6c2590ca2589029eed8e4d56ff008
MONAI __file__: D:\repos\tdt17_mini_project\venv\lib\site-packages\monai\__init__.py

Optional dependencies:
Pytorch Ignite version: 0.4.11
ITK version: NOT INSTALLED or UNKNOWN VERSION.
Nibabel version: 5.3.2
scikit-image version: 0.24.0
scipy version: 1.14.1
Pillow version: 11.0.0
Tensorboard version: 2.18.0
gdown version: NOT INSTALLED or UNKNOWN VERSION.
TorchVision version: 0.20.1+cu124
tqdm version: 4.67.0
lmdb version: NOT INSTALLED or UNKNOWN VERSION.
psutil version: 6.1.0
pandas version: NOT INSTALLED or UNKNOWN VERSION.
einops version: 0.8.0
transformers version: NOT INSTALLED or UNKNOWN VERSION.
mlflow version: NOT INSTALLED or UNKNOWN VERSION.
pynrrd version: NOT INSTALLED or UNKNOWN VERSION.
clearml version: NOT INSTALLED or UNKNOWN VERSION.

For details about installing the 

In [2]:
# Is the data isotropically spaced?
spacing = dataset[0]['image'].affine.diagonal()[:3].abs()
spacing  # all equal, so yes

tensor([1., 1., 1.], dtype=torch.float64)

In [3]:
import ipywidgets as widgets
from ipywidgets import interact
sample_idx = 10
sample = dataset[sample_idx]
image = monai.transforms.Orientation(axcodes="SPL")(sample['image'])
mask = monai.transforms.Orientation(axcodes="SPL")(sample['mask'])
def plot_slice(slice_idx):
    image_slice =  image[0,slice_idx]
    label0 = mask[0,slice_idx]
    label1 = mask[1,slice_idx]
    
    plt.figure(figsize=(8,8))
    plt.imshow(image_slice, cmap="gray", alpha=1.0)
    plt.imshow(label0, cmap="Reds", alpha=0.3)
    plt.imshow(label1, cmap="plasma", alpha=0.3)
    plt.axis("off")
    
    plt.show()
slider = widgets.IntSlider(value=mask.shape[1]//2, min=0, max=mask.shape[1]-1, step=1, description='Slice Index')
interact(plot_slice, slice_idx=slider);

interactive(children=(IntSlider(value=84, description='Slice Index', max=168), Output()), _dom_classes=('widge…

In [4]:
# Check data shapes
shape_tensor = torch.zeros(len(dataset), 4)
for i, data in enumerate(tqdm(dataset)):
    mask = data['mask']
    shape_tensor[i] = torch.tensor(mask.shape)

max_d = torch.zeros(4)
min_d = torch.zeros(4)
for d in range(shape_tensor.shape[1]):
    min_d[d] = shape_tensor[:, d].min()
    max_d[d] = shape_tensor[:, d].max()
    print(d, shape_tensor[:, d].shape)
min_d, max_d

100%|██████████| 130/130 [05:04<00:00,  2.35s/it]

0 torch.Size([130])
1 torch.Size([130])
2 torch.Size([130])
3 torch.Size([130])


(tensor([  2., 256., 240., 113.]), tensor([  2., 520., 520., 181.]))

In [ ]:
class_counts = np.array([0, 0, 0], dtype=np.float32)
for data in tqdm(dataset):
    mask = data['mask']
    counts = np.array([len(mask[mask == idx]) for idx in range(3)])  #/ len(dataset)
    class_counts += counts
class_counts

In [ ]:
n = class_counts[1:].sum()
c = 2
# Class weights
n / (c * class_counts)

In [ ]:
avg_mask = torch.zeros_like(dataset[0]['mask'])
for data in tqdm(dataset):
    avg_mask += data['mask']
avg_mask /= len(dataset)

In [ ]:
plt.imshow(avg_mask.cpu().mean(3).permute(1, 2, 0))
plt.axis('off')
plt.title("Average tumor pixels");

In [ ]:
plt.plot(torch.tensor([t.sum() for t in avg_mask[0].permute(2, 0, 1)]))
plt.xlabel("Z-level")
plt.ylabel("Tumor pixels")
plt.title("Average tumor Z-location")
plt.tight_layout();

In [ ]:
# Tumor percentage
total_pixels = torch.tensor(dataset[0]['mask'].shape).prod()
total_tumor = 0
total_non_tumor = 0
for data in tqdm(dataset):
    total_tumor += data['mask'].sum() / total_pixels
total_tumor /= len(dataset)
total_tumor

In [ ]:
max_ = 0
min_ = 1000
for data in tqdm(dataset):
    curr = data['image'].max()
    max_ = max_ if max_ >= curr else curr
    min_ = min_ if min_ <= curr else min_
min_, max_

In [ ]:
# Calculate intensity distribution
bincounts = torch.zeros(130, 256)
for i, data in enumerate(tqdm(dataset)):
    intensity_values = data['image'].flatten().divide(max_).multiply(255).round().int()
    bincount = torch.bincount(intensity_values, minlength=256)
    bincounts[i, :len(bincount)] = bincount

In [ ]:
x_extent = 255
mean = bincounts.mean(0)[:x_extent]
std = bincounts.std(0)[:x_extent]

plt.figure(figsize=(6, 6))
plt.bar(range(x_extent), mean, alpha=0.5, label='Mean Intensity Distribution', color='green')
# plt.fill_between(range(x_extent), mean - std, mean + std, color='lightblue', alpha=0.5, label='Standard Deviation')
# plt.yscale('symlog')
plt.xlabel('Intensity Value')
plt.ylabel('Frequency (Mean Count)')
plt.title('Intensity Distribution with Mean and Standard Deviation')
plt.legend()
plt.show()

In [ ]:
monai.visualize.matshow3d(monai.transforms.Orientation("SPL")(dataset[0]['image']), every_n=9, figsize=(6, 6))
plt.show()